In [9]:
import wave, struct

# framing
file = 'year2003-person00003-00013.wav'
waveFile = wave.open(file, mode='rb')
print(waveFile.getframerate())
print(waveFile.getnchannels())
print(waveFile.getsampwidth())
print(waveFile.getnframes())
f = waveFile.readframes(2)
data = struct.unpack("<h", f)
print(int(data[0]))

8000
1
1
64000
-32128


In [4]:
import noisereduce as nr
from scipy.io import wavfile
import IPython
import numpy as np

# load data
rate, data = wavfile.read("year2003-person00003-00013.wav")
# rate, data = wavfile.read("year2003-person00001-00013.wav")
print(len(data))
print(rate)

# # select section of data that is noise
# noisy_part = data[0000:4000].astype(float)
# data = data.astype(float)

# # perform noise reduction
# reduced_noise = nr.reduce_noise(audio_clip=data, noise_clip=noisy_part, verbose=True)

# # wavfile.write("example.wav", rate, reduced_noise.astype(np.int8))
# wavfile.write("example.wav", 4000, noisy_part)

64000
8000


In [ ]:
from logmmse import logmmse_from_file
from scipy.io import wavfile
import numpy as np

rate = 8000
out = logmmse_from_file('year2003-person00001-00013.wav')
wavfile.write("year2003-person00001-00013-improved.wav", rate, out.astype(np.int8))